In [135]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.random.seed(1)

In [136]:
# The shape of x_train and x_test is (6, 3) and  y_test and y_train is (6,). 
# Each vector has two features x1,x2 that belongs to one of 3 classes 
# using the (hot encoding) [1,0,0],[0,1,0] and [0,0,1] approximation
# If x1=x2 [1,0,0], x1<x2 [0,1,0] and x1>x2 [0,0,1].
X_train = np.array([[1,1],[3,4],[5,5],[9,7],[6,4],[4,4],[10,1],[1,3]]) 
Y_train = np.array([[1,0,0],[0,1,0],[1,0,0],[0,0,1],[0,0,1],[1,0,0],[0,0,1],[0,1,0]])
X_test = np.array([[3,3],[3,10],[1,5],[12,12],[2,2]])
Y_test = np.array([[1,0,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]])

In [137]:
print('shape Y_train=',Y_train.shape,'; shape Y_test=',Y_test.shape)
print('shape X_train=',X_train.shape,'; shape X_test=',X_test.shape)

shape Y_train= (8, 3) ; shape Y_test= (5, 3)
shape X_train= (8, 2) ; shape X_test= (5, 2)


In [138]:
# Initialization of the parameters through initialize() function. 
# The weights were chosen randomly using a normal distribution with gaussians.
# The transpose of x_train is initialized for future matrix operations. 
# The initial offset b = 0 and min_cost = 0.001 indirectly controls the number of iterations to 
# minimize the cost function by breaking the self-consistent loop when the cost function is lower than 0.001
# loc = mean of the normal distribution, size = shape of the output, scale = standard deviation
def initialize(x_train,y_train):
    std_gaussian=0.01
    alpha = -0.2
    min_cost = 0.001
    epoch=10000
    batch_size=4
    counter = 0
    w = np.random.normal(size = ([x_train.shape[1],y_train.shape[1]]),loc=0,scale=std_gaussian) 
    n = x_train.shape[0]
    x = np.transpose(x_train)
    b = np.zeros([y_train.shape[1]])
    iterations=int(n/batch_size) # automated - based on the batch size
    return w,x,n,b,alpha,min_cost,epoch,batch_size,counter,iterations

In [139]:
# Definition of activation function sigma(z) based on the sigmoid. 
# This function needs to be called by defining the type of activation function. i.e sigma(z,activation='sigmoid'). 
# This is important if we explore possible different activation functions in the next assignments.
def sigma(z_i,activation=False):
    if(activation==False):
        return print('Please choose an activation function')
    elif(activation=='sigmoid'):
        p_ik = 1/(1+np.exp(-z_i))
        return p_ik
    elif(activation=='softmax'):
        z_ik = np.exp(z_i)
        sum_row=np.sum(z_ik,axis=1).reshape(-1,1)
        p_ik=np.divide(z_ik,sum_row)
        return p_ik
    elif(activation=='relu'):
        p_ik = np.maximum(0.0,z_i)
        return p_ik

In [140]:
def relu_derivative(x):
    return np.where(np.maximum(0.0, x)>0,1,0)

In [141]:
def sigmoid_derivative(x):
    sig = sigma(x,activation=='sigmoid')
    sig_der = der*(1-der)
    return der

In [142]:
# This function calculates the cross-entropy loss function and the cost function. 
# Instead of using for-loops element-wise operations are performed with arrays. 
# You need to define which type of loss function to use: cost_function(y_train,p_i,loss_function='cross_entropy')
def cost_function(y_in,p_in,loss_function=False): # Calculate the loss and cost function using entering arrays
    if(loss_function==False):
        return print('Please choose a loss function')
    elif(loss_function=='cross_entropy'):
        nb = y_in.shape[0]
        loss_calc = -np.sum(y_in*np.log(p_in),axis=1)
        cost_calc = (1/nb)*np.sum(loss_calc)
        return loss_calc,cost_calc

In [143]:
        
# This function calculates the gradients and performs matrix operations to estimate the partial derivatives. 
# dLdb represents the derivative of the loss function with respect to the offset parameter b. 
# dJdb = derivative of the cost function with respect to b and dJdw = derivative of the cost 
# function with respect to a specific j-th weight. 
# Numpy library was used for vectorization and to perform element-wise operations instead of for loops.  
def softmax_backward(p_in,y_in,x_in):
    n_in = y_in.shape[0]
    dLdb = p_in - y_in
    dJdb = (1/n_in)*np.sum(p_in-y_in,axis=0)
    dJdw = (1/n)*np.dot(np.transpose(x_in),dLdb) # vector dJ/dW_j to update w_j
    return dJdw,dJdb

In [144]:
def sigmoid_backward(h_in,w_in,p_in,y_in,x_in):
    dLdb_vector=[]
    dLdw_vector=[]
    n_in = y_in.shape[0]
    der_sigmoid = h_in*(1-h_in)
    for i in range(n_in):
        dLdb = (p_in[i] - y_in[i])
        dLdb2 = np.matmul(w_in,dLdb)
        dLdb3 = dLdb2*der_sigmoid[i].T
        dLdb_vector.append(dLdb3)
        dLdb3=dLdb3.reshape(1,-1)
        x_reshape = np.reshape(x_in[i],(-1,1))
        dLdw1=(np.matmul(x_reshape,dLdb3))
        dLdw_vector.append(dLdw1)
    dldb_average = (1/n_in)*np.sum(dLdb_vector,axis=0)
    dldw_average = (1/n_in)*np.sum(dLdw_vector,axis=0)
   # print(dLdw_vector)
    return dldb_average,dldw_average

In [145]:
def relu_backward(h_in,w_in,p_in,y_in,x_in):
    dLdb_vector=[]
    dLdw_vector=[]
    n_in = y_in.shape[0]
    der_relu = relu_derivative(h_in)
    for i in range(n_in):
        dLdb = (p_in[i] - y_in[i])
        dLdb2 = np.matmul(w_in,dLdb)
        dLdb3 = dLdb2*der_relu[i].T
        dLdb_vector.append(dLdb3)
        dLdb3=dLdb3.reshape(1,-1)
        x_reshape = np.reshape(x_in[i],(-1,1))
        dLdw1=(np.matmul(x_reshape,dLdb3))
        dLdw_vector.append(dLdw1)
    dldb_average = (1/n_in)*np.sum(dLdb_vector,axis=0)
    dldw_average = (1/n_in)*np.sum(dLdw_vector,axis=0)
   # print(dLdw_vector)
    return dldb_average,dldw_average

In [146]:
# This function will randomly reshuffle X_train and Y_train with new indexes for the 
def random_batch(x_in,y_in):
    random_index = np.random.choice(x_in.shape[0],size = x_in.shape[0], replace= False)
    x_out = x_in[random_index]
    y_out = y_in[random_index]
    return x_out,y_out

In [148]:
def accuracy(x_acc,y_acc,w,b):
    z_acc = np.dot(x_acc,w)+b
    p_ik_acc=sigma(z_acc,activation='softmax') # probability using optimized parameters
    p_ik_max_acc= np.max(p_ik_acc,axis=1).reshape(-1,1) # find maximum for each row and reshape to column
    y_pred_acc=np.where(p_ik_acc>=p_ik_max_acc,1,0) # change p_ik to 1 in case the element of the row is >= maximum for row.
    acc=np.mean(y_pred_acc == y_acc) # Calculate the accuracy
    return acc,y_pred_acc

In [149]:
# Initialize w=weight, x=input vectors, n= number of input vectors 
# in the data set, alpha = learning rate, min_cost = control variable
w,x,n,b,alpha,min_cost,epoch,batch_size,counter,iterations=initialize(X_train,Y_train)

In [18]:
print('w=',w,'\nb=',b,'\n',w+b) 

w= [[ 0.01744812 -0.00761207  0.00319039]
 [-0.0024937   0.01462108 -0.02060141]] 
b= [0. 0. 0.] 
 [[ 0.01744812 -0.00761207  0.00319039]
 [-0.0024937   0.01462108 -0.02060141]]


In [19]:
z = np.dot(X_train,w)+b
print(z)
X_train,b

[[ 0.01495441  0.00700901 -0.01741102]
 [ 0.04236954  0.03564811 -0.07283446]
 [ 0.07477207  0.03504505 -0.08705508]
 [ 0.13957713  0.03383893 -0.11549633]
 [ 0.09471389  0.0128119  -0.06326328]
 [ 0.05981766  0.02803604 -0.06964406]
 [ 0.17198747 -0.06149961  0.0113025 ]
 [ 0.00996701  0.03625117 -0.05861383]]


(array([[ 1,  1],
        [ 3,  4],
        [ 5,  5],
        [ 9,  7],
        [ 6,  4],
        [ 4,  4],
        [10,  1],
        [ 1,  3]]), array([0., 0., 0.]))

In [20]:
z_ik = np.exp(z)
print(z_ik)

[[1.01506679 1.00703363 0.98273968]
 [1.04327994 1.03629112 0.92975473]
 [1.0776385  1.03566637 0.91662661]
 [1.14978749 1.03441798 0.89092384]
 [1.09934428 1.01289433 0.9386963 ]
 [1.06164294 1.02843275 0.93272575]
 [1.18766295 0.94035331 1.01136662]
 [1.01001684 1.03691626 0.94307088]]


In [21]:
sum_z=np.sum(z_ik,axis=1).reshape(-1,1)
print('sum_row=\n',sum_z)
p_ik=np.divide(z_ik,sum_z)
print('p_ik=\n',p_ik)

sum_row=
 [[3.0048401 ]
 [3.00932579]
 [3.02993147]
 [3.07512931]
 [3.0509349 ]
 [3.02280145]
 [3.13938288]
 [2.99000398]]
p_ik=
 [[0.33781058 0.33513718 0.32705224]
 [0.34668228 0.3443599  0.30895782]
 [0.35566431 0.34181181 0.30252387]
 [0.37389891 0.33638195 0.28971915]
 [0.3603303  0.33199474 0.30767497]
 [0.3512116  0.34022504 0.30856335]
 [0.37831096 0.29953445 0.32215459]
 [0.33779783 0.34679427 0.3154079 ]]


In [22]:
p_ik_max= np.max(p_ik,axis=1).reshape(-1,1) # find maximum for each row and reshape to column
p_ik_encoder=np.where(p_ik>=p_ik_max,1,0) # change p_ik to i
print(p_ik_encoder)

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]]


In [23]:
p_ik_act = sigma(z,activation='softmax') # debug softmax implementation
print(p_ik_act)

[[0.33781058 0.33513718 0.32705224]
 [0.34668228 0.3443599  0.30895782]
 [0.35566431 0.34181181 0.30252387]
 [0.37389891 0.33638195 0.28971915]
 [0.3603303  0.33199474 0.30767497]
 [0.3512116  0.34022504 0.30856335]
 [0.37831096 0.29953445 0.32215459]
 [0.33779783 0.34679427 0.3154079 ]]


In [24]:
p_ik_max= np.max(p_ik_act,axis=1).reshape(-1,1) # find maximum for each row and reshape to column
y_i=np.where(p_ik_act>=p_ik_max,1,0) # change p_ik to 1 in case the element of the row is >= maximum for row.
print(y_i) 

[[1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]]


In [25]:
y_i.shape,Y_train.shape # match between the shapes. Softmax implemented!
Y_train

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [26]:
p_ik-Y_train

array([[-0.66218942,  0.33513718,  0.32705224],
       [ 0.34668228, -0.6556401 ,  0.30895782],
       [-0.64433569,  0.34181181,  0.30252387],
       [ 0.37389891,  0.33638195, -0.71028085],
       [ 0.3603303 ,  0.33199474, -0.69232503],
       [-0.6487884 ,  0.34022504,  0.30856335],
       [ 0.37831096,  0.29953445, -0.67784541],
       [ 0.33779783, -0.65320573,  0.3154079 ]])

In [27]:
(1/n)*np.sum(p_ik-Y_train,axis=0)

array([-0.01978665,  0.08452992, -0.06474326])

In [28]:
(1/n)*np.dot(X_train.T,(p_ik-Y_train))

array([[ 0.5261256 ,  1.0999651 , -1.62609069],
       [ 0.04475321,  0.35063823, -0.39539144]])

In [29]:
## Implementations of the cost-function using cross-entropy loss function
print('Cost Function=',-(1/n)*np.sum(Y_train*np.log(p_ik)))
print('Loss Vector=',-np.sum(Y_train*np.log(p_ik),axis=1))

Cost Function= 1.1050967670314145
Loss Vector= [1.08526994 1.06606795 1.03376793 1.23884328 1.17871135 1.04636638
 1.13272374 1.05902355]


In [30]:
## Implemented function for calculating both loss function for each x_i and cost function as a sum over all features.
# Function call to estimate the cost function based on the cross-entropy loss function and 
# print both the cost function and the loss function. 
# l = loss function, j = cost function
l,j= cost_function(Y_train,p_ik,loss_function='cross_entropy')
print('Cost Function =',j,'\nLoss Function vector =',l)

Cost Function = 1.1050967670314145 
Loss Function vector = [1.08526994 1.06606795 1.03376793 1.23884328 1.17871135 1.04636638
 1.13272374 1.05902355]


In [32]:
# Function call to estimate the gradients dw = dJdw and db = dJdb. 
# These values are used to update the parameter of the model.
dw,db= softmax_backward(p_ik,Y_train,X_train)
print(dw,db)

[[ 0.5261256   1.0999651  -1.62609069]
 [ 0.04475321  0.35063823 -0.39539144]] [-0.01978665  0.08452992 -0.06474326]


In [33]:
# First update of the weight and offset parameters.
w += alpha*dw
b += alpha*db

In [34]:
print('w =',w,'\n b =',b)

w = [[-0.087777   -0.22760509  0.32840853]
 [-0.01144435 -0.05550657  0.05847688]] 
 b = [ 0.00395733 -0.01690598  0.01294865]


In [35]:
# Test split of the matrix for the minibatch
for i in range(iterations): # loop to split the matrix during the loops.
    print(X_train[i*batch_size:(i+1)*batch_size,:],'\n')

[[1 1]
 [3 4]
 [5 5]
 [9 7]] 

[[ 6  4]
 [ 4  4]
 [10  1]
 [ 1  3]] 



In [36]:
# Definition of a counter and threshold of min_cost = 0.001 for convergence. 
# This means that the parameters of the model are optimized. 
# All the previous operations are performed inside the loop until the break of the self-consistent while-loop.
#while(j>min_cost):
for E in range(epoch):
    X_train_minibatch,Y_train_minibatch = random_batch(X_train,Y_train)
    for i in range(iterations):
        x_loop = X_train_minibatch[i*batch_size:(i+1)*batch_size,:]
        y_loop = Y_train_minibatch[i*batch_size:(i+1)*batch_size,:]
            # print(y_loop,i) debug ok
        z = np.dot(x_loop,w)+b
            # print(z,i)      debug ok 
        p_ik=sigma(z,activation='softmax')
            # print(p_ik,i)   debug ok
        l,j= cost_function(y_loop,p_ik,loss_function='cross_entropy')
            # print(l,j,i)    debug ok
            # print('Cost Function =',j,'\n') debug ok
        dw,db= softmax_backward(p_ik,y_loop,x_loop)
            # print(dw,db,i)   debug ok
        w +=alpha*dw
        b +=alpha*db
        counter+=1
    print('Epoch =',E,'; Cost Function =',j,'\n')
print('Number of iterations = ',counter)

Epoch = 0 ; Cost Function = 1.4191580088433828 

Epoch = 1 ; Cost Function = 1.309625556676689 

Epoch = 2 ; Cost Function = 1.1624032304000258 

Epoch = 3 ; Cost Function = 0.6931196521982571 

Epoch = 4 ; Cost Function = 0.8190897978777469 

Epoch = 5 ; Cost Function = 0.8800270917784795 

Epoch = 6 ; Cost Function = 1.363060804742208 

Epoch = 7 ; Cost Function = 0.5266546034444421 

Epoch = 8 ; Cost Function = 0.6165276870988984 

Epoch = 9 ; Cost Function = 1.9171664238314896 

Epoch = 10 ; Cost Function = 1.1023648495860707 

Epoch = 11 ; Cost Function = 0.9570900414982249 

Epoch = 12 ; Cost Function = 0.6531032573988704 

Epoch = 13 ; Cost Function = 0.8076301090235948 

Epoch = 14 ; Cost Function = 1.2584745698537698 

Epoch = 15 ; Cost Function = 0.8033153595158389 

Epoch = 16 ; Cost Function = 0.7262524776086718 

Epoch = 17 ; Cost Function = 0.8415215517553241 

Epoch = 18 ; Cost Function = 1.0182297284510666 

Epoch = 19 ; Cost Function = 0.6848745920555112 

Epoch = 20 ;

Epoch = 237 ; Cost Function = 0.20039668377100484 

Epoch = 238 ; Cost Function = 0.16085521340780376 

Epoch = 239 ; Cost Function = 0.16243850208494834 

Epoch = 240 ; Cost Function = 0.05809396740184866 

Epoch = 241 ; Cost Function = 0.21516833424510318 

Epoch = 242 ; Cost Function = 0.056370272695345794 

Epoch = 243 ; Cost Function = 0.18025376136366902 

Epoch = 244 ; Cost Function = 0.17108063511258317 

Epoch = 245 ; Cost Function = 0.13509143585338299 

Epoch = 246 ; Cost Function = 0.24305358553619605 

Epoch = 247 ; Cost Function = 0.22499377768771653 

Epoch = 248 ; Cost Function = 0.13180180452561086 

Epoch = 249 ; Cost Function = 0.1755898708245433 

Epoch = 250 ; Cost Function = 0.09252320799867997 

Epoch = 251 ; Cost Function = 0.2267244922660803 

Epoch = 252 ; Cost Function = 0.23949151309605626 

Epoch = 253 ; Cost Function = 0.05449303797781656 

Epoch = 254 ; Cost Function = 0.10869150477795536 

Epoch = 255 ; Cost Function = 0.2195647415619267 

Epoch = 256 ; 

Epoch = 437 ; Cost Function = 0.08903590939237391 

Epoch = 438 ; Cost Function = 0.06073393763560264 

Epoch = 439 ; Cost Function = 0.11154801642347834 

Epoch = 440 ; Cost Function = 0.10900314981864667 

Epoch = 441 ; Cost Function = 0.06437762411135953 

Epoch = 442 ; Cost Function = 0.1279720629362823 

Epoch = 443 ; Cost Function = 0.1398948559481555 

Epoch = 444 ; Cost Function = 0.11219562190650131 

Epoch = 445 ; Cost Function = 0.12338935590495892 

Epoch = 446 ; Cost Function = 0.10804314011485701 

Epoch = 447 ; Cost Function = 0.12802164840449348 

Epoch = 448 ; Cost Function = 0.12153621708511526 

Epoch = 449 ; Cost Function = 0.14001891126759863 

Epoch = 450 ; Cost Function = 0.07228789284714941 

Epoch = 451 ; Cost Function = 0.13401796430415344 

Epoch = 452 ; Cost Function = 0.14947408326240355 

Epoch = 453 ; Cost Function = 0.12064516268461775 

Epoch = 454 ; Cost Function = 0.07733367953990808 

Epoch = 455 ; Cost Function = 0.12813916544886597 

Epoch = 456 ; 

Epoch = 737 ; Cost Function = 0.08268624530887667 

Epoch = 738 ; Cost Function = 0.05533590368461826 

Epoch = 739 ; Cost Function = 0.09756328110889868 

Epoch = 740 ; Cost Function = 0.03330426873289073 

Epoch = 741 ; Cost Function = 0.06800431555209245 

Epoch = 742 ; Cost Function = 0.06479392578020608 

Epoch = 743 ; Cost Function = 0.04622645023689729 

Epoch = 744 ; Cost Function = 0.06376268895275966 

Epoch = 745 ; Cost Function = 0.019266350274500188 

Epoch = 746 ; Cost Function = 0.04073742733015645 

Epoch = 747 ; Cost Function = 0.01905252719840034 

Epoch = 748 ; Cost Function = 0.06843904501030798 

Epoch = 749 ; Cost Function = 0.08996939917828713 

Epoch = 750 ; Cost Function = 0.08617758140515194 

Epoch = 751 ; Cost Function = 0.08172935761841765 

Epoch = 752 ; Cost Function = 0.018648724780519002 

Epoch = 753 ; Cost Function = 0.06949670250507423 

Epoch = 754 ; Cost Function = 0.0817833736602603 

Epoch = 755 ; Cost Function = 0.03232171027927949 

Epoch = 756

Epoch = 937 ; Cost Function = 0.05554799184122812 

Epoch = 938 ; Cost Function = 0.0755368645707603 

Epoch = 939 ; Cost Function = 0.05692405099842345 

Epoch = 940 ; Cost Function = 0.05699980561857829 

Epoch = 941 ; Cost Function = 0.039010857932537134 

Epoch = 942 ; Cost Function = 0.038216566500953925 

Epoch = 943 ; Cost Function = 0.06296757975389272 

Epoch = 944 ; Cost Function = 0.0671265370222061 

Epoch = 945 ; Cost Function = 0.03048445771618894 

Epoch = 946 ; Cost Function = 0.03871700156979657 

Epoch = 947 ; Cost Function = 0.037056446992622574 

Epoch = 948 ; Cost Function = 0.026670852552722245 

Epoch = 949 ; Cost Function = 0.06619863458779299 

Epoch = 950 ; Cost Function = 0.02577781032295049 

Epoch = 951 ; Cost Function = 0.060106171708990346 

Epoch = 952 ; Cost Function = 0.03960872124698427 

Epoch = 953 ; Cost Function = 0.05515214223668148 

Epoch = 954 ; Cost Function = 0.04619466340522377 

Epoch = 955 ; Cost Function = 0.06494943425702786 

Epoch = 9

...

Epoch = 9031 ; Cost Function = 0.005052611816971641 

Epoch = 9032 ; Cost Function = 0.0033178826776693853 

Epoch = 9033 ; Cost Function = 0.006410029504266864 

Epoch = 9034 ; Cost Function = 0.007964902924360532 

Epoch = 9035 ; Cost Function = 0.0014901079223019096 

Epoch = 9036 ; Cost Function = 0.007591455214768656 

Epoch = 9037 ; Cost Function = 0.00627484962840123 

Epoch = 9038 ; Cost Function = 0.00331525528030077 

Epoch = 9039 ; Cost Function = 0.0037802961660824974 

Epoch = 9040 ; Cost Function = 0.007588020597149758 

Epoch = 9041 ; Cost Function = 0.005702372602055769 

Epoch = 9042 ; Cost Function = 0.0033281371672089725 

Epoch = 9043 ; Cost Function = 0.005613882163593464 

Epoch = 9044 ; Cost Function = 0.006846706990459742 

Epoch = 9045 ; Cost Function = 0.0026040637537404574 

Epoch = 9046 ; Cost Function = 0.0037259457431739703 

Epoch = 9047 ; Cost Function = 0.005608401882243468 

Epoch = 9048 ; Cost Function = 0.0052958842087224055 

Epoch = 9049 ; Cost Fun

Epoch = 9232 ; Cost Function = 0.007791662650047471 

Epoch = 9233 ; Cost Function = 0.0025740363143275337 

Epoch = 9234 ; Cost Function = 0.005590853925294644 

Epoch = 9235 ; Cost Function = 0.005589626985075098 

Epoch = 9236 ; Cost Function = 0.005573662678186672 

Epoch = 9237 ; Cost Function = 0.004411838413674049 

Epoch = 9238 ; Cost Function = 0.007430269831221408 

Epoch = 9239 ; Cost Function = 0.0014377014377147611 

Epoch = 9240 ; Cost Function = 0.005109055845279819 

Epoch = 9241 ; Cost Function = 0.006438013431990809 

Epoch = 9242 ; Cost Function = 0.004526028950964769 

Epoch = 9243 ; Cost Function = 0.0067877661407392265 

Epoch = 9244 ; Cost Function = 0.007781252800356466 

Epoch = 9245 ; Cost Function = 0.005106127244846832 

Epoch = 9246 ; Cost Function = 0.0036895244404571616 

Epoch = 9247 ; Cost Function = 0.003800177298174998 

Epoch = 9248 ; Cost Function = 0.007777605259895828 

Epoch = 9249 ; Cost Function = 0.0014585790091754631 

Epoch = 9250 ; Cost Fun

Epoch = 9431 ; Cost Function = 0.005005776745663693 

Epoch = 9432 ; Cost Function = 0.004906634774285666 

Epoch = 9433 ; Cost Function = 0.004842707353577227 

Epoch = 9434 ; Cost Function = 0.0042792307412335695 

Epoch = 9435 ; Cost Function = 0.004319747052548094 

Epoch = 9436 ; Cost Function = 0.005315633247533741 

Epoch = 9437 ; Cost Function = 0.0037582448517162396 

Epoch = 9438 ; Cost Function = 0.006590483448112935 

Epoch = 9439 ; Cost Function = 0.004998040829541045 

Epoch = 9440 ; Cost Function = 0.003037828705917573 

Epoch = 9441 ; Cost Function = 0.003184854014699821 

Epoch = 9442 ; Cost Function = 0.0071037532663357 

Epoch = 9443 ; Cost Function = 0.002492881087025254 

Epoch = 9444 ; Cost Function = 0.006555895416162149 

Epoch = 9445 ; Cost Function = 0.003183428151976838 

Epoch = 9446 ; Cost Function = 0.0067082516929181345 

Epoch = 9447 ; Cost Function = 0.0037547075394638928 

Epoch = 9448 ; Cost Function = 0.004171389297187545 

Epoch = 9449 ; Cost Functi

Epoch = 9584 ; Cost Function = 0.006495117702004784 

Epoch = 9585 ; Cost Function = 0.006461055013694126 

Epoch = 9586 ; Cost Function = 0.004757105214188335 

Epoch = 9587 ; Cost Function = 0.005915705052929727 

Epoch = 9588 ; Cost Function = 0.003132890620855338 

Epoch = 9589 ; Cost Function = 0.00699505707719749 

Epoch = 9590 ; Cost Function = 0.004991465642388343 

Epoch = 9591 ; Cost Function = 0.0069932703385361195 

Epoch = 9592 ; Cost Function = 0.0035152378506380384 

Epoch = 9593 ; Cost Function = 0.001892120956859858 

Epoch = 9594 ; Cost Function = 0.008709328875503405 

Epoch = 9595 ; Cost Function = 0.00543019511012683 

Epoch = 9596 ; Cost Function = 0.001383801584867095 

Epoch = 9597 ; Cost Function = 0.004823480349644629 

Epoch = 9598 ; Cost Function = 0.002633411084938803 

Epoch = 9599 ; Cost Function = 0.004241452090943029 

Epoch = 9600 ; Cost Function = 0.0066389974194378375 

Epoch = 9601 ; Cost Function = 0.0031300443907912685 

Epoch = 9602 ; Cost Functi

Epoch = 9880 ; Cost Function = 0.003986546071974699 

Epoch = 9881 ; Cost Function = 0.0062657775185683716 

Epoch = 9882 ; Cost Function = 0.006445812527489431 

Epoch = 9883 ; Cost Function = 0.005737279619224089 

Epoch = 9884 ; Cost Function = 0.002900130608583063 

Epoch = 9885 ; Cost Function = 0.0052720791135840595 

Epoch = 9886 ; Cost Function = 0.0028982463019193256 

Epoch = 9887 ; Cost Function = 0.0046143060613320595 

Epoch = 9888 ; Cost Function = 0.006261387148157471 

Epoch = 9889 ; Cost Function = 0.002881707843854298 

Epoch = 9890 ; Cost Function = 0.006440847703330811 

Epoch = 9891 ; Cost Function = 0.005851047274351699 

Epoch = 9892 ; Cost Function = 0.006781436502374293 

Epoch = 9893 ; Cost Function = 0.006004154158352697 

Epoch = 9894 ; Cost Function = 0.003579861605656915 

Epoch = 9895 ; Cost Function = 0.005131109475301478 

Epoch = 9896 ; Cost Function = 0.002880133591807252 

Epoch = 9897 ; Cost Function = 0.003982322469557198 

Epoch = 9898 ; Cost Func

In [48]:
p_ik_max= np.max(p_ik,axis=1).reshape(-1,1) # find maximum for each row and reshape to column
y_id=np.where(p_ik>=p_ik_max,1,0) # change p_ik to 1 in case the element of the row is >= maximum for row.
print(y_loop)

[[0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]]


In [49]:
Y_train

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [46]:
print('Accuracy = ',np.mean(y_i==Y_train))

Accuracy =  0.6666666666666666


In [40]:
# Calculate the prediction over the test dataset 
z_test = np.dot(X_test,w)+b
p_ik_test=sigma(z_test,activation='softmax') # probability using optimized parameters
p_ik_test

array([[9.94437412e-01, 4.58421904e-03, 9.78369394e-04],
       [7.63557674e-28, 1.00000000e+00, 2.66793512e-46],
       [3.28189293e-15, 1.00000000e+00, 1.17350940e-27],
       [6.41420065e-01, 3.29215888e-03, 3.55287776e-01],
       [9.94983366e-01, 4.53231782e-03, 4.84315919e-04]])

In [41]:
p_ik_max_test= np.max(p_ik_test,axis=1).reshape(-1,1) # find maximum for each row and reshape to column
y_i_test=np.where(p_ik_test>=p_ik_max_test,1,0) # change p_ik to 1 in case the element of the row is >= maximum for row.
print(y_i_test)

[[1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]]


In [53]:
np.mean(y_i_test == Y_test) # Calculate the accuracy

1.0

In [54]:
accuracy(X_test,Y_test,w,b) # Test accuracy function

(1.0, array([[1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0]]))

In [150]:
class hidden_layer:
    Count = 0
    np.random.seed(10)
    alpha = -0.2
    def __init__(self, x_train, y_train='False', n_nodes='False', activation='False'):
            self.x_train = x_train
            self.y_train = y_train
            self.n_nodes = n_nodes
            self.activation = activation
            self.w=[]
            self.b=[]
            hidden_layer.Count += 1
  
    def add_random_parameters(self):
        if (self.activation == "softmax"):
            self.w = np.random.normal(size = ([self.x_train.shape[1],self.y_train.shape[1]]),loc=0,scale=0.01) 
            self.b = np.zeros([self.y_train.shape[1]])
        else:
            self.w = np.random.normal(size = ([self.x_train.shape[1],self.n_nodes]),loc=0,scale=0.01) 
            self.b = np.zeros([self.n_nodes])
        
    def forward(self,x):
        z_new = np.dot(x,self.w)+self.b
        h_new = sigma(z_new,activation=self.activation)
        return z_new,h_new
    
    def update_parameters(self,dw,db):
        self.w+=alpha*dw
        self.b+=alpha*db
        
    def Weight(self):
        print ("w=",self.w)
        
    def Bias(self):
        print ("b=",self.b)
    
    def parameters(self):
        weight = self.w
        bias = self.b
        return weight,bias
    
#    class create: # subclass
#        countt=0
#        def test(activation):
#            if (activation == "softmax"):
#                return "softmax"

In [367]:
# loop to create dictionaries with values.
#layers = {}
#for i in range(hidden_layer.Count+1):
#    layers.update({'layer'+str(i): w+i})
    

In [151]:
layer1 = hidden_layer(X_train,n_nodes=2,activation="sigmoid") # start new layer with 4 nodes

In [152]:
layer1.add_random_parameters() # start values for parameters weight and bias

In [153]:
# check the previous started parameters
layer1.Weight()
layer1.Bias() 

w= [[ 1.33158650e-02  7.15278974e-03]
 [-1.54540029e-02 -8.38384993e-05]]
b= [0. 0.]


In [154]:
# calculate the linear combination matrix for all data and activation matrix using sigmoid activation
z1,h1=layer1.forward(X_train)
print(h1)

[[0.49946547 0.50176723]
 [0.49453311 0.50528056]
 [0.49732735 0.50883527]
 [0.50291616 0.51594165]
 [0.50451967 0.51064374]
 [0.49786188 0.50706848]
 [0.52939224 0.51785342]
 [0.49173922 0.50172531]]


In [155]:
# create a second layer with matrix N (number of features) x M (Number of nodes in the layer) from input h1
# and only one output node
layer2 = hidden_layer(h1,Y_train,activation="softmax")

In [156]:
layer2.add_random_parameters() # start weights and bias

In [157]:
layer2.Weight() # check weights and bias
layer2.Bias()

w= [[ 6.21335974e-03 -7.20085561e-03  2.65511586e-03]
 [ 1.08548526e-03  4.29143093e-05 -1.74600211e-03]]
b= [0. 0. 0.]


In [158]:
layer2.x_train

array([[0.49946547, 0.50176723],
       [0.49453311, 0.50528056],
       [0.49732735, 0.50883527],
       [0.50291616, 0.51594165],
       [0.50451967, 0.51064374],
       [0.49786188, 0.50706848],
       [0.52939224, 0.51785342],
       [0.49173922, 0.50172531]])

In [159]:
z2,h2=layer2.forward(h1) # feedforward

In [160]:
print(h2) # print 
h2.shape

[[0.33449178 0.33208443 0.33342379]
 [0.33448399 0.33209745 0.33341856]
 [0.33449079 0.33209053 0.33341868]
 [0.33450441 0.33207669 0.33341891]
 [0.33450515 0.33207211 0.33342274]
 [0.33449104 0.332089   0.33341996]
 [0.33455519 0.3320085  0.33343631]
 [0.33447718 0.33210437 0.33341845]]


(8, 3)

In [161]:
layer2.Bias()
layer2.Weight()

b= [0. 0. 0.]
w= [[ 6.21335974e-03 -7.20085561e-03  2.65511586e-03]
 [ 1.08548526e-03  4.29143093e-05 -1.74600211e-03]]


In [162]:
z1.shape,h1.shape,

((8, 2), (8, 2))

In [163]:
z2.shape,h2.shape

((8, 3), (8, 3))

In [164]:
X_train.shape

(8, 2)

In [165]:
l,j= cost_function(Y_train,h2,loss_function='cross_entropy')
print('Cost Function =',j,'\nLoss Function vector =',l)

Cost Function = 1.09813375363183 
Loss Function vector = [1.09514298 1.10232683 1.09514592 1.0983556  1.0983441  1.09514518
 1.09830342 1.102306  ]


In [166]:
w1,b1 = layer1.parameters()
w2,b2 = layer2.parameters()

In [167]:
w2,b2

(array([[ 6.21335974e-03, -7.20085561e-03,  2.65511586e-03],
        [ 1.08548526e-03,  4.29143093e-05, -1.74600211e-03]]),
 array([0., 0., 0.]))

In [168]:
dw2,db2= softmax_backward(h2,Y_train,h1)
print(dw2,db2)

[[-0.01883923  0.0434916  -0.02465237]
 [-0.0195689   0.04303206 -0.02346316]] [-0.04050006  0.08207788 -0.04157783]


In [299]:
(h2-Y_train)[0,:]

array([-0.66550822,  0.33208443,  0.33342379])

In [199]:
w2

array([[ 6.21335974e-03, -7.20085561e-03,  2.65511586e-03],
       [ 1.08548526e-03,  4.29143093e-05, -1.74600211e-03]])

In [200]:
#print(w2,w1)
ggg=np.dot((h2-Y_train),w2.T)
ggg

array([[-0.00564106, -0.00129031],
       [ 0.007773  , -0.00024773],
       [-0.00564112, -0.0012903 ],
       [-0.00208269,  0.0015412 ],
       [-0.00208264,  0.0015412 ],
       [-0.0056411 , -0.0012903 ],
       [-0.00208184,  0.00154122],
       [ 0.00777291, -0.00024774]])

In [204]:
fl=h1[0,:]*(1-h1[0,:])
fl
f=h1*(1-h1)
f

array([[0.24999971, 0.24999688],
       [0.24997011, 0.24997212],
       [0.24999286, 0.24992194],
       [0.2499915 , 0.24974586],
       [0.24997957, 0.24988671],
       [0.24999543, 0.24995004],
       [0.2491361 , 0.24968126],
       [0.24993176, 0.24999702]])

In [301]:
g=np.dot(w2,(h2-Y_train)[0,:])
g

array([-0.00564106, -0.00129031])

In [206]:
gg=np.multiply(fl,g)
gg

array([-0.00141026, -0.00032257])

In [253]:
test=np.multiply(f,ggg)
test

array([[-1.41026219e-03, -3.22572679e-04],
       [ 1.94301871e-03, -6.19267549e-05],
       [-1.41023941e-03, -3.22473953e-04],
       [-5.20654809e-04,  3.84908934e-04],
       [-5.20618047e-04,  3.85124486e-04],
       [-1.41024994e-03, -3.22510717e-04],
       [-5.18660837e-04,  3.84814807e-04],
       [ 1.94269749e-03, -6.19346486e-05]])

In [303]:
gg.shape
gg=gg.reshape(1,-1)
gg

array([[-0.00141026, -0.00032257]])

In [255]:
(1/8)*np.sum(test,axis=0),db1

(array([-2.38121129e-04,  7.92868433e-06]),
 array([-2.38121129e-04,  7.92868433e-06]))

In [256]:
X_train.T

array([[ 1,  3,  5,  9,  6,  4, 10,  1],
       [ 1,  4,  5,  7,  4,  4,  1,  3]])

In [257]:
(1/8)*np.matmul(X_train.T,test)

array([[-0.00241586,  0.0007688 ],
       [-0.0008435 ,  0.00012015]])

In [296]:
#implement
#n=2
#i=1
#dLdb[2]=(h2-Y_train)
#ggg=np.dot(dLdb[n],w[i+1].T)
#f=h1*(1-h1)
#test=np.multiply(f,ggg)
#(1/8)*np.sum(test,axis=0),db1
#(1/8)*np.matmul(X_train.T,test)

IndexError: list assignment index out of range

In [258]:
dw1

array([[-0.00241586,  0.0007688 ],
       [-0.0008435 ,  0.00012015]])

In [259]:
db1

array([-2.38121129e-04,  7.92868433e-06])

In [231]:
X_train[0].shape

(2,)

In [232]:
(np.matmul(np.reshape(X_train[0],(-1,1)),gg))

array([[-0.00141026, -0.00032257],
       [-0.00141026, -0.00032257]])

In [128]:
#debug
#w1,db1,r,resul= grad_hidden(h1[0,:],w2,h2[0,:],Y_train[0,:],X_train[0,:])
#print(dw1,'\n\n',db1,'\n\n',r,'\n\n',resul)

NameError: name 'db2' is not defined

In [236]:
db1,dw1= sigmoid_backward(h1,w2,h2,Y_train,X_train)

In [237]:
db1

array([-2.38121129e-04,  7.92868433e-06])

In [235]:
dw1

array([[-0.00241586,  0.0007688 ],
       [-0.0008435 ,  0.00012015]])

In [123]:
layer2.update_parameters(dw2,db2)
layer1.update_parameters(dw1,db1)

NameError: name 'dw2' is not defined

In [1979]:
layer1.Bias()
layer2.Bias()

b= [ 1.42872677e-04 -4.75721060e-06]
b= [ 0.02430004 -0.04924673  0.0249467 ]


In [1980]:
layer1.Weight()
layer2.Bias()

w= [[ 0.01476538  0.00669151]
 [-0.0149479  -0.00015593]]
b= [ 0.02430004 -0.04924673  0.0249467 ]


In [1]:
# Loop over dictionaries
#layers = {0:'layer0', 1:'layer1', 2:'layer2',3:'layer3'}
#for i in range(jsneural.Count):
#    x=eval(layers[i]+'.h')
#    print(x)